<a href="https://colab.research.google.com/github/worldstar0722/IS_4487_25FA/blob/main/assignment_06_data_cleaning_Choi_Ellie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 4487 Assignment 6: Data Cleaning with Airbnb Listings

In this assignment, you will:
- Load a raw Airbnb listings dataset
- Identify and resolve missing or inconsistent data
- Decide what data to drop, keep, or clean
- Save a clean dataset to use in Assignment 7

## Why This Matters

Data cleaning is one of the most important steps in any analysis — but it's often the least visible. Airbnb hosts, managers, and policy teams rely on clean data to make decisions. This assignment gives you experience cleaning raw data and justifying your choices so others can understand your process.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Assignments/assignment_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Dataset Description

The dataset you'll be using is a **detailed Airbnb listing file**, available from [Inside Airbnb](https://insideairbnb.com/get-the-data/).

Each row represents one property listing. The columns include:

- **Host attributes** (e.g., host ID, host name, host response time)
- **Listing details** (e.g., price, room type, minimum nights, availability)
- **Location data** (e.g., neighborhood, latitude/longitude)
- **Property characteristics** (e.g., number of bedrooms, amenities, accommodates)
- **Calendar/booking variables** (e.g., last review date, number of reviews)

📌 The schema is consistent across cities, so you can expect similar columns regardless of the location you choose.


## 1. Choose a City & Upload Your Dataset

📥 Follow these steps:

1. Go to: [https://insideairbnb.com/get-the-data/](https://insideairbnb.com/get-the-data/)
2. Choose a city you’re interested in.
3. Download the file named: **`listings.csv.gz`** under that city.
4. In your notebook:
   - Open the left sidebar
   - Click the folder icon 📁
   - Click the upload icon ⬆️ and choose your `listings.csv.gz` file
5. Use the file path `/content/listings.csv.gz` when loading your data.
6. Import standard libraries (`pandas`, `numpy`, `seaborn`, `matplotlib`)


In [ ]:
# Import necessary libraries 🔧
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load your uploaded file (path "/content/listings.csv.gz") 🔧
file_path = "/content/listings.csv.gz"
df = pd.read_csv(file_path, low_memory=False)

print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/listings.csv.gz'

## 2. Explore Missing Values

Business framing:

Stakeholders don’t like surprises in the data. Missing values can break dashboards, confuse pricing models, or create blind spots for host managers.

Explore how complete your dataset is:
- Count the null values of each column
- Create visuals (e.g. heatmaps, boxplots, bar charts, etc) to help show what columns are missing values
- Keep in mind which column(s) are missing too much data, you will delete these in the next step

### In your markdown:
1. What are the top 3 columns with the most missing values?
2. Which ones are likely to create business issues?
3. Which could be safely ignored or dropped?



In [ ]:
# Add code here 🔧

missing_counts = df.isnull().sum().sort_values(ascending=False)

print("Top 10 columns with missing values:")
print(missing_counts.head(10))

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.barplot(x=missing_counts.head(10).values, y=missing_counts.head(10).index, palette="viridis")
plt.title("Top 10 Columns with Most Missing Values")
plt.xlabel("Number of Missing Values")
plt.ylabel("Columns")
plt.show()

### ✍️ Your Response: 🔧

1. The top 3 columns with the most missing values are calendar_updated (12,608 missing), neighbourhood_group_cleansed (12,608 missing), and host_neighbourhood (7,335 missing). </br>

2. host_neighbourhood could create business issues because location context is important for pricing models, neighborhood trend analysis, and marketing strategies. If this is missing, it reduces accuracy in location-based insights.</br>

3. calendar_updated and neighbourhood_group_cleansed could be safely ignored or dropped since they are entirely missing and provide no business value. Keeping them would only clutter the dataset without improving decision-making.


## 3. Drop Columns That Aren’t Useful

Business framing:  

Not every column adds value. Analysts often remove columns that are too empty, irrelevant, or repetitive — especially when preparing data for others.

Make a decision:

- Choose 2–4 columns to drop from your dataset
- Document your reasons for each one
- Confirm they're gone with `.head()` or `.info()`

### In Your Response:
1. Which columns did you drop?
2. Why were they not useful from a business perspective?
3. What could go wrong if you left them in?



In [ ]:
# Add code here 🔧
cols_to_drop = ["calendar_updated", "neighbourhood_group_cleansed", "host_about"]

df_cleaned = df.drop(columns=cols_to_drop)

print("Remaining columns after drop:", df_cleaned.shape[1])

### ✍️ Your Response: 🔧
1.I dropped the columns calendar_updated, neighbourhood_group_cleansed, and host_about.

2.These columns were not useful from a business perspective because:

- calendar_updated was entirely empty.

- neighbourhood_group_cleansed was also completely missing.

- host_about contained long free-text responses with many missing values, making it less reliable for structured analysis.

3.If I left them in, it could confuse analysts, inflate processing time, and clutter dashboards with irrelevant or incomplete information.



## 4. Fill or Fix Values in Key Columns

Business framing:  

Let’s say your manager wants to see a map of listings with prices and review scores. If key fields are blank, the map won’t work. But not all missing values should be filled the same way.

- Choose 2 columns with missing values
- Use a strategy to fill or flag those values
  - (e.g., median, “unknown”, forward-fill, or a placeholder)
- Explain what you did and why

### In your response:
1. What two columns did you clean?
2. What method did you use for each, and why?
3. What risks are there in how you filled the data?

In [ ]:
# Your code for converting column data types 🔧

df_cleaned["reviews_per_month"] = df_cleaned["reviews_per_month"].fillna(0)

df_cleaned["host_response_time"] = df_cleaned["host_response_time"].fillna("unknown")

df_cleaned[["reviews_per_month", "host_response_time"]].head(10)


### ✍️ Your Response: 🔧
1. I cleaned the columns reviews_per_month and host_response_time.</br>

2. For reviews_per_month, I filled missing values with 0, because if no reviews exist, it makes sense to treat that as zero instead of leaving it blank.
For host_response_time, I replaced missing values with “unknown” so that the dataset remains consistent and still usable for categorical analysis.</br>

3. The risks are that filling missing values can hide some uncertainty. For example, assuming 0 reviews might overlook cases where data was not collected, and labeling response time as “unknown” groups different host behaviors into one category.

## 5. Convert and Clean Data Types

Business framing:  

Sometimes columns that look like numbers are actually stored as text — which breaks calculations and slows down analysis. Common examples are price columns with dollar signs or availability stored as strings.

- Identify one column with the wrong data type
- Clean and convert it into a usable format (e.g., from string to number)
- Check your work by summarizing or plotting the cleaned column

### In Your Response: :
1. What column did you fix?
2. What cleaning steps did you apply?
3. How does this help prepare the data for later use?


In [ ]:
# Clean or adjust your dataset 🔧

print("Before:", df_cleaned["price"].dtype)

df_cleaned["price"] = (
    df_cleaned["price"]
    .replace('[\$,]', '', regex=True)
    .astype(float)
)

print("After:", df_cleaned["price"].dtype)

df_cleaned["price"].describe()


### ✍️ Your Response: 🔧
1. I fixed the price column. </br>

2. I cleaned it by removing the dollar signs ($) and commas, then converted the column from string (object) to float so it can be used in calculations.</br>

3. This helps prepare the data for later use because price is one of the most important variables in Airbnb analysis — converting it to numeric allows us to calculate averages, run comparisons, and build models without errors.

## 6. Remove Duplicate Records

Business framing:  

If a listing appears twice, it could inflate revenue estimates or confuse users. Airbnb needs each listing to be unique and accurate.

- Check for rows that are exact duplicates
- If your data has an ID column and each ID is supposed to unique, then make sure there are no duplicate IDs
- Remove duplicates if found

### In your markdown:
1. Did you find duplicates?
2. How did you decide what to drop or keep?
3. Why are duplicates risky for Airbnb teams?


In [ ]:
# Add code here 🔧

duplicate_rows = df_cleaned.duplicated().sum()
print("Duplicate rows:", duplicate_rows)

df_cleaned = df_cleaned.drop_duplicates()

duplicate_ids = df_cleaned["id"].duplicated().sum()
print("Duplicate IDs:", duplicate_ids)

df_cleaned = df_cleaned.drop_duplicates(subset="id", keep="first")

print("Shape after removing duplicates:", df_cleaned.shape)


### ✍️ Your Response: 🔧 🔧
1. I checked for duplicates and found some duplicate records based on rows and IDs.</br>

2. I kept the first occurrence of each listing and removed duplicates to ensure each id represents a unique property.</br>

3. Duplicates are risky for Airbnb teams because they can inflate metrics like revenue, listing counts, and host activity. This could mislead pricing models, confuse hosts, and damage trust in analytics dashboards.

## 7. Export Cleaned Data

Before wrapping up, export your cleaned Airbnb dataset to a CSV file. You'll need this file for **Assignment 7**, where you'll perform data transformation techniques.

Make sure your data has:
- Cleaned and consistent column values
- Proper data types for each column
- Any unnecessary columns removed

This file should be the version of your dataset that you’d feel confident sharing with a teammate or using for deeper analysis.



```
# Explanation:
# - "cleaned_airbnb_data.csv" is the name of the file that will be saved
# - index=False prevents pandas from writing row numbers into the CSV
# - The file will be saved to your working directory (in Colab, you'll need to download it manually. Once you see the data in your files tab, just click on the three dots, then click “download”)
# - YOU MAY NEED TO PRESS “RUN” MULTIPLE TIMES IN ORDER FOR IT TO SHOW UP
# - FOR SOME DEVICES, IT MAY TAKE A FEW MINUTES BEFORE YOUR FILE SHOWS UP

```





In [ ]:
# export csv here 🔧

df_cleaned.to_csv("cleaned_airbnb_data.csv", index=False)

print("File saved as cleaned_airbnb_data.csv")


## 8. Final Reflection

You’ve just cleaned a real-world Airbnb dataset — the kind of work that happens every day in analyst and data science roles.

Before you move on to data transformation in Assignment 7, take a few moments to reflect on the decisions you made and what you learned.

### In your markdown:
1. What was the most surprising or challenging part of cleaning this dataset?
2. How did you decide which data to drop, fix, or keep?
3. What’s one way a business team (e.g., hosts, pricing analysts, platform ops) might benefit from the cleaned version of this data?
4. If you had more time, what would you explore or clean further?
5. How does this relate to your customized learning outcome you created in canvas?


Write your response clearly in full sentences. No more than a few sentences required per response.


### ✍️ Your Response: 🔧
1. The most surprising part of cleaning this dataset was how many columns had large amounts of missing data, especially calendar_updated and neighbourhood_group_cleansed, which were completely empty.</br>

2. I decided which data to drop, fix, or keep by considering both the percentage of missing values and whether the column had business value. For example, host_neighbourhood was important to keep, but calendar_updated was not.</br>

3. A pricing analyst team could benefit from the cleaned dataset because accurate and complete price and reviews_per_month fields help improve revenue forecasting and pricing models.</br>

4. If I had more time, I would explore cleaning free-text fields like host_about and neighborhood_overview, as they might provide useful insights with natural language processing. </br>

5. This relates to my customized learning outcome because I wanted to get better at preparing raw datasets for real-world analysis, and this assignment helped me practice structured decision-making in data cleaning.